In [18]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import os

class SingleStepImageEnv(gym.Env):
    """
    Entorno con una sola decisión:
      - Observación = 2 imágenes apiladas en un solo tensor
      - Acciones: 2 (0 -> escoger imagen A, 1 -> escoger imagen B)
      - Recompensas: 
          * Si action=0, reward=10 con prob 0.75
          * Si action=1, reward=10 con prob 0.25
      - Episodio termina en un solo paso.
    """
    def __init__(self, imageA, imageB, rewardA=10.0, rewardB=10.0, probA=0.75, probB=0.25):
        super().__init__()
        # Se asume imageA e imageB son np.array con shape (C, H, W)
        assert imageA.shape == imageB.shape, "Ambas imágenes deben tener la misma forma"
        self.imageA = imageA
        self.imageB = imageB
        self.rewardA = rewardA
        self.rewardB = rewardB
        self.probA = probA
        self.probB = probB

        # Creamos la observación apilada
        # Apilamos en el eje de canales: obs_shape = (2*C, H, W)
        self.observation = np.concatenate([imageA, imageB], axis=0)
        obs_shape = self.observation.shape  # (2*C, H, W)

        # Espacios de Gym
        self.action_space = spaces.Discrete(2)  # 2 acciones: A=0, B=1
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=obs_shape, dtype=np.float32)

        self.done = False

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.done = False
        info = {}
        return self.observation, info

    def step(self, action):
        if self.done:
            raise RuntimeError("Episodio ya terminado. Llama reset().")

        if action == 0:  # Acción A
            reward = self.rewardA if np.random.rand() < self.probA else 0.0
        else:            # Acción B
            reward = self.rewardB if np.random.rand() < self.probB else 0.0

        self.done = True
        return self.observation, reward, self.done, False, {}


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNActorCritic(nn.Module):
    """
    CNN que produce logits (para 2 acciones) y un valor (escalar).
    Supongamos la entrada es shape (batch_size, 2*C, H, W).
    """
    def __init__(self, in_channels=2, num_actions=2):
        super().__init__()

        # in_channels=2 si son 2 imágenes en gris
        # Si fuese RGB, in_channels=6, etc.

        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)

        # Calcula tamaño tras convoluciones: depende de H, W
        self.fc1   = nn.Linear(32*6*6, 128)  # <-- Ajustar si la imagen final es 7x7, depende del input
        self.actor = nn.Linear(128, num_actions)
        self.critic= nn.Linear(128, 1)

    def forward(self, x):
        # x shape: (batch_size, in_channels, H, W)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        # Flatten
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))

        logits = self.actor(x)
        value  = self.critic(x)
        return logits, value


In [20]:
import torch
from torch.distributions import Categorical
import torch.optim as optim
import numpy as np

def compute_returns_and_advantages(rewards, values, gamma=1.0):
    returns = []
    advantages = []
    for r, v in zip(rewards, values):
        G = r
        A = G - v
        returns.append(G)
        advantages.append(A)
    return returns, advantages

def ppo_train(env, policy_net, optimizer, epochs=10, episodes_per_epoch=100, gamma=1.0, epsilon=0.2, entropy_coef=0.01):
    policy_net.train()

    for epoch in range(epochs):
        states, actions, rewards, log_probs_old, values_old = [], [], [], [], []

        for _ in range(episodes_per_epoch):
            obs, _ = env.reset()  # obs shape: (2*C, H, W)
            obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)  # (1, 2*C, H, W)

            logits, value = policy_net(obs_tensor)
            dist = Categorical(logits=logits)
            action = dist.sample()
            log_prob = dist.log_prob(action)

            next_obs, reward, done, truncated, info = env.step(action.item())

            states.append(obs)  # guardamos la imagen apilada
            actions.append(action.item())
            rewards.append(reward)
            log_probs_old.append(log_prob.item())
            values_old.append(value.item())

        # Calcular returns y ventajas
        returns, advantages = compute_returns_and_advantages(rewards, values_old, gamma=gamma)

        # Convertir a tensores
        states_tensor       = torch.tensor(states, dtype=torch.float32)
        # (batch_size, 2*C, H, W)
        actions_tensor      = torch.tensor(actions, dtype=torch.long)
        old_log_probs_tensor = torch.tensor(log_probs_old, dtype=torch.float32)
        returns_tensor      = torch.tensor(returns, dtype=torch.float32)
        advantages_tensor   = torch.tensor(advantages, dtype=torch.float32)

        # Forward batch
        # States_tensor shape: (batch_size, 2*C, H, W)
        logits, values = policy_net(states_tensor)
        dist = Categorical(logits=logits)
        log_probs = dist.log_prob(actions_tensor)

        ratio = torch.exp(log_probs - old_log_probs_tensor)
        ratio_clipped = torch.clamp(ratio, 1.0 - epsilon, 1.0 + epsilon)

        policy_loss_1 = ratio * advantages_tensor
        policy_loss_2 = ratio_clipped * advantages_tensor
        policy_loss   = -torch.mean(torch.min(policy_loss_1, policy_loss_2))

        value_loss = torch.mean((values.squeeze() - returns_tensor)**2)
        entropy = dist.entropy().mean()

        loss = policy_loss + 0.5 * value_loss - entropy_coef * entropy

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_reward = np.mean(rewards)
        print(f"Epoch {epoch+1}/{epochs}: Loss={loss.item():.3f} PolicyLoss={policy_loss.item():.3f} ValueLoss={value_loss.item():.3f} Entropy={entropy.item():.3f} AvgReward={avg_reward:.2f}")

def main(image_a=None, image_b=None, reward_a=10.0, reward_b=10.0, prob_a=0.75, prob_b=0.25, checkpoint='ppo_checkpoint.pth', model_save_path='ppo_checkpoint.pth'):
    imageA = image_a # np.random.rand(1, 28, 28).astype(np.float32)  
    imageB = image_b # np.random.rand(1, 28, 28).astype(np.float32)

    env = SingleStepImageEnv(imageA, imageB, rewardA=reward_a, rewardB=reward_b, probA=prob_a, probB=prob_b)

    # in_channels=2 si es gris; =6 si es RGB (2 imágenes x 3 canales cada una)
    policy_net = CNNActorCritic(in_channels=2, num_actions=2)

    optimizer = optim.Adam(policy_net.parameters(), lr=1e-3)


    if checkpoint is not None and os.path.exists(checkpoint):
        print(f"Cargando modelo desde {checkpoint}")
        checkpoint = torch.load(checkpoint)
        policy_net.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(f"Modelo y optimizador cargados correctamente.")
    else:
        print("Entrenando desde cero.")

    ppo_train(env, policy_net, optimizer,
              epochs=10, episodes_per_epoch=100,
              gamma=1.0, epsilon=0.2, entropy_coef=0.01)
    
    # Guardar modelo
    torch.save({
        "model_state_dict": policy_net.state_dict(),
        "optimizer_state_dict": optimizer.state_dict()
    }, model_save_path)

    print("Modelo guardado en", checkpoint)

    # Evaluación
    policy_net.eval()
    test_rewards = []
    for _ in range(1000):
        obs, _ = env.reset()
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            logits, value = policy_net(obs_tensor)
            dist = Categorical(logits=logits)
            action = dist.sample().item()
        _, reward, done, truncated, info = env.step(action)
        test_rewards.append(reward)

    print(f"\nRecompensa promedio en 1000 episodios de test: {np.mean(test_rewards):.2f}")


In [21]:
# Preprocesamiento de imágenes
import cv2

def preprocess_image(image):
    # load image
    image = cv2.imread(image)
    # convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # resize to 28x28
    resized = cv2.resize(gray, (28, 28), interpolation=cv2.INTER_AREA)
    # normalize
    normalized = resized / 255.0

    # convert to tensor
    tensor = torch.tensor(normalized, dtype=torch.float32).unsqueeze(0)
    #tensor = np.asarray(tensor)
    return tensor

In [22]:
image_a = preprocess_image("/Users/mtazc/Documents/AI_projects/test_ppo_rl/apple.jpg")
image_b = preprocess_image("/Users/mtazc/Documents/AI_projects/test_ppo_rl/cabbage.jpg")
image_c = preprocess_image("/Users/mtazc/Documents/AI_projects/test_ppo_rl/rock.jpg")
image_d = preprocess_image("/Users/mtazc/Documents/AI_projects/test_ppo_rl/stick.jpg")

In [23]:
image_a.shape

torch.Size([1, 28, 28])

In [24]:
main(image_a, image_b)

Entrenando desde cero.
Epoch 1/10: Loss=18.525 PolicyLoss=-4.627 ValueLoss=46.319 Entropy=0.690 AvgReward=4.70
Epoch 2/10: Loss=21.117 PolicyLoss=-5.314 ValueLoss=52.876 Entropy=0.691 AvgReward=5.60
Epoch 3/10: Loss=22.202 PolicyLoss=-5.650 ValueLoss=55.718 Entropy=0.692 AvgReward=6.10
Epoch 4/10: Loss=16.877 PolicyLoss=-4.132 ValueLoss=42.033 Entropy=0.693 AvgReward=4.80
Epoch 5/10: Loss=15.401 PolicyLoss=-3.641 ValueLoss=38.098 Entropy=0.693 AvgReward=4.60
Epoch 6/10: Loss=15.543 PolicyLoss=-3.665 ValueLoss=38.429 Entropy=0.692 AvgReward=5.00
Epoch 7/10: Loss=15.264 PolicyLoss=-3.589 ValueLoss=37.718 Entropy=0.688 AvgReward=5.40
Epoch 8/10: Loss=13.756 PolicyLoss=-2.901 ValueLoss=33.328 Entropy=0.675 AvgReward=5.30
Epoch 9/10: Loss=13.521 PolicyLoss=-3.178 ValueLoss=33.411 Entropy=0.638 AvgReward=6.30
Epoch 10/10: Loss=11.982 PolicyLoss=-2.693 ValueLoss=29.360 Entropy=0.567 AvgReward=6.70
Modelo guardado en ppo_checkpoint.pth

Recompensa promedio en 1000 episodios de test: 6.49


In [25]:
main(image_c, image_d, reward_a=1.0, reward_b=1.0, prob_a=0.75, prob_b=0.25, checkpoint='ppo_checkpoint.pth', model_save_path='c_d_ppo_checkpoint.pth')

Cargando modelo desde ppo_checkpoint.pth
Modelo y optimizador cargados correctamente.
Epoch 1/10: Loss=18.615 PolicyLoss=5.166 ValueLoss=26.906 Entropy=0.400 AvgReward=0.67
Epoch 2/10: Loss=19.583 PolicyLoss=5.322 ValueLoss=28.528 Entropy=0.296 AvgReward=0.71
Epoch 3/10: Loss=17.507 PolicyLoss=4.986 ValueLoss=25.046 Entropy=0.253 AvgReward=0.75
Epoch 4/10: Loss=14.609 PolicyLoss=4.479 ValueLoss=20.264 Entropy=0.233 AvgReward=0.72
Epoch 5/10: Loss=11.445 PolicyLoss=3.866 ValueLoss=15.161 Entropy=0.234 AvgReward=0.69
Epoch 6/10: Loss=8.374 PolicyLoss=3.189 ValueLoss=10.374 Entropy=0.245 AvgReward=0.71
Epoch 7/10: Loss=5.746 PolicyLoss=2.510 ValueLoss=6.477 Entropy=0.264 AvgReward=0.77
Epoch 8/10: Loss=4.121 PolicyLoss=2.008 ValueLoss=4.232 Entropy=0.290 AvgReward=0.72
Epoch 9/10: Loss=2.744 PolicyLoss=1.511 ValueLoss=2.474 Entropy=0.317 AvgReward=0.74
Epoch 10/10: Loss=1.835 PolicyLoss=1.117 ValueLoss=1.444 Entropy=0.349 AvgReward=0.73
Modelo guardado en {'model_state_dict': OrderedDict(

In [26]:
main(image_b, image_c, reward_a=10.0, reward_b=1.0, prob_a=0.25, prob_b=0.75, checkpoint='c_d_ppo_checkpoint.pth', model_save_path='b_c_ppo_checkpoint.pth')

Cargando modelo desde c_d_ppo_checkpoint.pth
Modelo y optimizador cargados correctamente.
Epoch 1/10: Loss=6.468 PolicyLoss=-0.564 ValueLoss=14.072 Entropy=0.454 AvgReward=1.84
Epoch 2/10: Loss=7.850 PolicyLoss=-1.285 ValueLoss=18.280 Entropy=0.476 AvgReward=2.35
Epoch 3/10: Loss=8.862 PolicyLoss=-1.722 ValueLoss=21.179 Entropy=0.492 AvgReward=2.63
Epoch 4/10: Loss=7.655 PolicyLoss=-1.445 ValueLoss=18.210 Entropy=0.501 AvgReward=2.24
Epoch 5/10: Loss=5.054 PolicyLoss=-0.736 ValueLoss=11.589 Entropy=0.508 AvgReward=1.45
Epoch 6/10: Loss=8.053 PolicyLoss=-1.699 ValueLoss=19.515 Entropy=0.514 AvgReward=2.35
Epoch 7/10: Loss=9.101 PolicyLoss=-2.041 ValueLoss=22.294 Entropy=0.517 AvgReward=2.65
Epoch 8/10: Loss=7.099 PolicyLoss=-1.446 ValueLoss=17.099 Entropy=0.517 AvgReward=2.03
Epoch 9/10: Loss=6.065 PolicyLoss=-1.170 ValueLoss=14.481 Entropy=0.514 AvgReward=1.74
Epoch 10/10: Loss=7.828 PolicyLoss=-1.637 ValueLoss=18.939 Entropy=0.510 AvgReward=2.20
Modelo guardado en {'model_state_dict':

In [27]:
def evaluate_model(image1, image2, reward1, reward2, prob1, prob2, checkpoint):

    env = SingleStepImageEnv(image1, image2, rewardA=reward1, rewardB=reward2, probA=prob1, probB=prob2)

    # load model
    policy_net = CNNActorCritic(in_channels=2, num_actions=2)

    if checkpoint is not None and os.path.exists(checkpoint):
        print(f"Cargando modelo desde {checkpoint}")
        checkpoint = torch.load(checkpoint)
        policy_net.load_state_dict(checkpoint['model_state_dict'])
        print(f"Modelo y optimizador cargados correctamente.")
    else:
        print("Checkpoint no encontrado, la evaluación no se puede realizar.")
        return -1 
    policy_net.eval()
    test_rewards = []
    for _ in range(1000):
        obs, _ = env.reset()
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            logits, value = policy_net(obs_tensor)
            dist = Categorical(logits=logits)
            action = dist.sample().item()
        _, reward, done, truncated, info = env.step(action)
        test_rewards.append(reward)

    print(f"\nRecompensa promedio en 1000 episodios de test: {np.mean(test_rewards):.2f}")

In [29]:
evaluate_model(image_b, image_c, 10.0, 1.0, 0.25, 0.75, 'b_c_ppo_checkpoint.pth')

Cargando modelo desde b_c_ppo_checkpoint.pth
Modelo y optimizador cargados correctamente.

Recompensa promedio en 1000 episodios de test: 2.11
